### Detach one from another

In [1]:
import trimesh
import numpy as np
from utils import *

In [2]:
mesh = trimesh.load("data/two_spheres2.ply")

Split the mesh into connected components

In [3]:
connected_components = mesh.split(only_watertight=True)

Compute centroids of each component

In [4]:
centroids = [comp.centroid for comp in connected_components]

Compute the vector between the centroids

In [5]:
translation_vector = centroids[1] - centroids[0]
distance = np.linalg.norm(translation_vector)
normalized_vector = translation_vector / distance

Shift the second component along the normalized vector

In [ ]:
connected_components[1].apply_translation(normalized_vector * 0.7)

<trimesh.Trimesh(vertices.shape=(476, 3), faces.shape=(948, 3), name=`two_spheres2.ply`)>

In [7]:
new_mesh = trimesh.util.concatenate(connected_components)

In [8]:
x = pymesh.form_mesh(vertices=new_mesh.vertices, faces = new_mesh.faces)

In [9]:
visualize_two_meshes(mesh, x)

Widget(value='<iframe src="http://localhost:37609/index.html?ui=P_0x7f3b9e3a10f0_0&reconnect=auto" class="pyvi…

None

## Detach mesh

In [2]:
import trimesh
import numpy as np
from utils import *

In [ ]:
### Generating a sample mesh
"""
mesh = trimesh.load("data/shrink.ply")
connected_components = mesh.split(only_watertight=True)
centroids = [comp.centroid for comp in connected_components]
translation_vector = centroids[1] - centroids[0]
distance = np.linalg.norm(translation_vector)
normalized_vector = translation_vector / distance
connected_components[1].apply_translation(normalized_vector * 0.09)
new_mesh = trimesh.util.concatenate(connected_components)
x = pymesh.form_mesh(vertices=new_mesh.vertices, faces = new_mesh.faces)
print(pymesh.detect_self_intersection(x))
pymesh.save_mesh("data/shrink2.ply", x)
"""


[[ 828  104]
 [ 839  104]
 [ 471 1203]
 [ 471  989]
 [ 839  471]
 [1204  471]
 [ 256  989]
 [ 622  989]
 [ 622 1355]
 [ 171 1365]
 [ 538 1365]
 [ 622  999]
 [ 622 1365]
 [ 363 1365]
 [ 548 1365]
 [1000  548]
 [ 998  548]
 [1364  548]
 [1376  363]
 [1010  548]
 [1376  548]
 [  43 1289]
 [ 615 1289]
 [ 460 1195]
 [ 410 1289]
 [  23  829]
 [ 408  829]
 [  66  829]
 [ 388  829]
 [ 388 1195]
 [  43 1376]
 [ 546 1376]
 [ 615 1376]
 [ 460  828]
 [ 818   23]
 [ 820   23]
 [1185   23]
 [ 189  932]
 [ 556  932]
 [ 189 1289]
 [1294  189]
 [1294  556]
 [ 928  189]
 [ 929  189]
 [1295  189]
 [1174   23]
 [1103  380]
 [1174  380]
 [ 553  929]
 [  13  737]
 [  13 1103]
 [ 386  737]
 [1291  178]
 [ 920  178]
 [1291  553]
 [ 738   12]
 [ 738  386]
 [1104  386]
 [1100   12]
 [ 379 1100]
 [ 545  920]
 [ 187  920]
 [ 379  734]
 [ 187 1286]
 [  19  734]
 [1101   19]
 [ 735  374]
 [1101  374]
 [1187  374]
 [ 174  916]
 [ 550  916]
 [ 550 1286]
 [ 909  174]
 [1283  174]
 [1274  174]
 [ 905  166]
 [ 905  174]

In [11]:
import trimesh
import open3d as o3d
import numpy as np
from utils import *

In [26]:
def trimesh_to_open3d(tri_mesh: trimesh.Trimesh) -> o3d.geometry.TriangleMesh:
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(tri_mesh.vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(tri_mesh.faces)
    return o3d_mesh

def pymesh_to_trimesh(mesh):
    return trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)

def trimesh_to_pymesh(mesh):
    return pymesh.form_mesh(vertices=mesh.vertices, faces=mesh.faces)

def open3d_to_pymesh(o3d_mesh: o3d.geometry.TriangleMesh) -> pymesh.Mesh:
    vertices = np.asarray(o3d_mesh.vertices)
    faces = np.asarray(o3d_mesh.triangles)
    return  pymesh.form_mesh(vertices, faces)


def open3d_to_trimesh(o3d_mesh: o3d.geometry.TriangleMesh) -> trimesh.Trimesh:
    vertices = np.asarray(o3d_mesh.vertices)
    faces = np.asarray(o3d_mesh.triangles)
    return trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

In [13]:
def detach_inner_component(
    input_path,
    contact_threshold=0.001,
    num_sample_points=50000
):
    """
    1) Use trimesh to read and split (connected components).
    2) Convert each submesh to Open3D geometry.
    3) Push the inner mesh inward where it contacts the outer mesh.
    """
    # 1) Read mesh with trimesh and split into multiple components
    tm = trimesh.load(input_path, process=False)
    submeshes = tm.split(only_watertight=True) 
    
    # 2) Sort by volume
    submeshes_sorted = sorted(submeshes, key=lambda m: m.volume)
    mesh_inner_tm = submeshes_sorted[0]
    mesh_outer_tm = submeshes_sorted[-1]
    
    # 3) Convert to Open3D
    mesh_inner = trimesh_to_open3d(mesh_inner_tm)
    mesh_outer = trimesh_to_open3d(mesh_outer_tm)
    
    # For visualization and processing later
    mesh_inner.compute_vertex_normals()
    mesh_outer.compute_vertex_normals()

    # Sample outer for distance checks
    # Generate num_sample_points uniformly distributed points on the surface of mesh_outer. This helps in efficiently computing the nearest neighbors.
    pcd_outer = mesh_outer.sample_points_poisson_disk(number_of_points=num_sample_points)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_outer)

    inner_vertices = np.asarray(mesh_inner.vertices)
    center_inner = np.mean(inner_vertices, axis=0)

    # For each vertex, find distance to outer sphere
    for i in range(len(inner_vertices)):
        v = inner_vertices[i]
        # k: # of nearest neighors found (should be 1)
        # idx: Index of the nearest neighbor (outersurface)
        # dist_sq: Squared distance to the nearst neighbor
        k, idx, dist_sq = pcd_tree.search_knn_vector_3d(v, 1) # 1 -> We want a nearest neighbor
        if k > 0:
            closest_point = np.asarray(pcd_outer.points)[idx[0]]
            dist = np.linalg.norm(v - closest_point)
            print(dist)
            if dist < contact_threshold:
                # shrink inward by (contact_threshold - dist)
                offset = contact_threshold - dist
                direction = v - center_inner
                r_len = np.linalg.norm(direction)
                if r_len > 1e-12:
                    direction_unit = direction / r_len
                    inner_vertices[i] = v - offset * direction_unit


    mesh_inner.vertices = o3d.utility.Vector3dVector(inner_vertices)

    # Merge the two meshes
    combined_mesh = mesh_inner + mesh_outer
    return combined_mesh


In [18]:
input_ply = "data/shrink.ply"
final = detach_inner_component(input_path=input_ply, contact_threshold=0.06, num_sample_points=50000)

0.15175096273322966
0.10258809670862686
0.1258986124530101
0.10971344920490403
0.06343990219656172
0.08563083208217252
0.09316353585804608
0.041532243246916636
0.06518310862392182
0.11362517259168285
0.08809606057211189
0.03832025074340903
0.0591786475734117
0.09885768623296227
0.11410398117471818
0.0728673453090295
0.050384453632835904
0.04444642185309562
0.0224549434210638
0.031552410611137956
0.036009819486971184
0.007632451645306626
0.024661930503170358
0.03853289643282022
0.050560836837174855
0.02688511639113346
0.01785384801281095
0.06591587211136582
0.032013888546200374
0.06450218565840841
0.012237617238040854
0.007507861360144227
0.0048834738214490565
0.007388773188300871
0.028803700461891363
0.009888850710365979
0.06582614295455966
0.037082044769931835
0.012411750935859948
0.007024760588948744
0.018481313432665385
0.005140263388668046
0.008311856840992435
0.012916975551870599
0.021852057839994634
0.033529471268734456
0.16762050965062492
0.20111878648278544
0.2043995363176561
0

In [19]:
original = pymesh.load_mesh("data/shrink.ply")
final = open3d_to_pymesh(final)
print(pymesh.detect_self_intersection(original))
print(pymesh.detect_self_intersection(final))

[[ 465  844]
 [ 465 1199]
 [ 465  830]
 [ 465 1156]
 [ 828  465]
 [1154  465]
 [1211  465]
 [ 864  465]
 [1232  465]
 [ 825  465]
 [1194  465]]
[]


In [20]:
visualize_two_meshes(original, final)

Widget(value='<iframe src="http://localhost:37613/index.html?ui=P_0x7fd5cee5a950_3&reconnect=auto" class="pyvi…

None

In [16]:
tri_original = pymesh_to_trimesh(original)
tri_final = pymesh_to_trimesh(final)

In [17]:
original_submeshes = tri_original.split(only_watertight=True)
original_inner = sorted(original_submeshes, key=lambda m: m.volume)[0]
final_submeshes = tri_final.split(only_watertight=True)
final_inner = sorted(final_submeshes, key=lambda m: m.volume)[0]

In [18]:
evaluation(original_inner, final_inner)

+-----------------------------------+-----------+-----------+
| Metric                            |    Before |     After |
+===================================+===========+===========+
| Number of vertices                | 368       | 368       |
+-----------------------------------+-----------+-----------+
| Number of faces                   | 732       | 732       |
+-----------------------------------+-----------+-----------+
| Number of intersecting face pairs |   0       |   0       |
+-----------------------------------+-----------+-----------+
| Volume                            |   1.27715 |   1.25232 |
+-----------------------------------+-----------+-----------+
| Area                              |   5.7148  |   5.64798 |
+-----------------------------------+-----------+-----------+
| Intact vertices (%)               | nan       |  85.0543  |
+-----------------------------------+-----------+-----------+


## Other case

In [ ]:
def shrink_inner_mesh_any_case(
    inner_o3d_mesh: o3d.geometry.TriangleMesh,
    outer_o3d_mesh: o3d.geometry.TriangleMesh,
    contact_threshold=0.06,
    max_iters=10,
    num_sample_points=50000
):
    """
    Iteratively shrinks inner mesh so it lies inside outer mesh

    1) sample the outer mesh into a point cloud.
    2) convert the outer mesh to a Trimesh and use `trimesh.contains(points)` to see which points are inside (True) or outside (False).
    3) For each iteration:
       a) Find which vertices of inner mesh are outside -> pull them inward.
       b) For vertices that are inside but too close to the outer surface,
          also pull them inward slightly.
    4) Stop early if no vertex changed in an iteration.
    """

    # Build a point cloud from the outer mesh + KD-tree
    pcd_outer = outer_o3d_mesh.sample_points_poisson_disk(number_of_points=num_sample_points)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_outer)
    
    # Convert the outer mesh to Trimesh for inside/outside checks
    outer_tri = open3d_to_trimesh(outer_o3d_mesh)

    # Access the inner mesh vertices
    inner_verts = np.asarray(inner_o3d_mesh.vertices)
    center_inner = np.mean(inner_verts, axis=0)

    for _iter in range(max_iters):
        changed_any = False

        # Check which vertices are inside vs. outside
        #    `outer_tri.contains(pts)` -> array of True/False
        inside_mask = outer_tri.contains(inner_verts)
        
        # Iterate over each vertex
        for i in range(len(inner_verts)):
            v = inner_verts[i]
            if not inside_mask[i]:
                # This vertex is outside -> pull inward
                direction = v - center_inner
                length = np.linalg.norm(direction)
                if length > 1e-12:
                    direction_unit = direction / length
                    # Move half the threshold inward each iteration
                    move_amount = 0.5 * contact_threshold
                    inner_verts[i] = v - move_amount * direction_unit
                changed_any = True
                continue

            # If inside, check distance to nearest surface point
            k, idx, dist_sq = pcd_tree.search_knn_vector_3d(v, 1)
            if k > 0:
                dist = np.sqrt(dist_sq[0])
                if dist < contact_threshold:
                    # Pull inward so that it is at least contact_threshold from outer surface
                    offset = contact_threshold - dist
                    direction = v - center_inner
                    length = np.linalg.norm(direction)
                    if length > 1e-12:
                        direction_unit = direction / length
                        inner_verts[i] = v - offset * direction_unit
                        changed_any = True

        if not changed_any:
            # No vertex changed => we've converged
            break

    inner_o3d_mesh.vertices = o3d.utility.Vector3dVector(inner_verts)
    return inner_o3d_mesh



def detach_inner_component_general(
    input_path,
    contact_threshold=0.06,
    max_iters=10,
    num_sample_points=50000
):
    """
    1) Load a mesh and split into submeshes.
    2) Identify smallest-volume piece as 'inner', largest-volume piece as 'outer'.
    3) Convert each to Open3D. 
    4) Iteratively shrink the inner mesh so it stays inside the outer.
    """
    # Load & split the mesh
    tm = trimesh.load(input_path, process=False)
    submeshes = tm.split(only_watertight=True)
    
    # Sort by volume: smallesr -> "inner", larger -> "outer"
    submeshes_sorted = sorted(submeshes, key=lambda m: m.volume)
    mesh_inner_tm = submeshes_sorted[0]
    mesh_outer_tm = submeshes_sorted[-1]

    # Convert to Open3D
    mesh_inner_o3d = trimesh_to_open3d(mesh_inner_tm)
    mesh_outer_o3d = trimesh_to_open3d(mesh_outer_tm)

    # Compute normals for better sampling
    mesh_inner_o3d.compute_vertex_normals()
    mesh_outer_o3d.compute_vertex_normals()

    # Shrink the inner mesh
    mesh_inner_o3d = shrink_inner_mesh_any_case(
        inner_o3d_mesh=mesh_inner_o3d,
        outer_o3d_mesh=mesh_outer_o3d,
        contact_threshold=contact_threshold,
        max_iters=max_iters,
        num_sample_points=num_sample_points
    )

    # combine them for 
    combined = mesh_inner_o3d + mesh_outer_o3d
    return combined


In [ ]:
input_ply = "data/shrink2.ply"

final2 = detach_inner_component_general(
    input_path=input_ply,
    contact_threshold=0.06,
    max_iters=10
)

original2 = pymesh.load_mesh("data/shrink2.ply")
final2 = open3d_to_pymesh(final2)
print(pymesh.detect_self_intersection(original2))
print(pymesh.detect_self_intersection(final2))

[[ 828  104]
 [ 839  104]
 [ 471 1203]
 [ 471  989]
 [ 839  471]
 [1204  471]
 [ 256  989]
 [ 622  989]
 [ 622 1355]
 [ 171 1365]
 [ 538 1365]
 [ 622  999]
 [ 622 1365]
 [ 363 1365]
 [ 548 1365]
 [1000  548]
 [ 998  548]
 [1364  548]
 [1376  363]
 [1010  548]
 [1376  548]
 [  43 1289]
 [ 615 1289]
 [ 460 1195]
 [ 410 1289]
 [  23  829]
 [ 408  829]
 [  66  829]
 [ 388  829]
 [ 388 1195]
 [  43 1376]
 [ 546 1376]
 [ 615 1376]
 [ 460  828]
 [ 818   23]
 [ 820   23]
 [1185   23]
 [ 189  932]
 [ 556  932]
 [ 189 1289]
 [1294  189]
 [1294  556]
 [ 928  189]
 [ 929  189]
 [1295  189]
 [1174   23]
 [1103  380]
 [1174  380]
 [ 553  929]
 [  13  737]
 [  13 1103]
 [ 386  737]
 [1291  178]
 [ 920  178]
 [1291  553]
 [ 738   12]
 [ 738  386]
 [1104  386]
 [1100   12]
 [ 379 1100]
 [ 545  920]
 [ 187  920]
 [ 379  734]
 [ 187 1286]
 [  19  734]
 [1101   19]
 [ 735  374]
 [1101  374]
 [1187  374]
 [ 174  916]
 [ 550  916]
 [ 550 1286]
 [ 909  174]
 [1283  174]
 [1274  174]
 [ 905  166]
 [ 905  174]

In [35]:
visualize_two_meshes(original2, final2)

Widget(value='<iframe src="http://localhost:37613/index.html?ui=P_0x7fd5396e5600_5&reconnect=auto" class="pyvi…

None